In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3316332/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_22_9_0,0.998964,0.789149,0.989720,0.995508,0.992481,0.006927,1.409960,0.031377,0.011341,0.021359,0.341904,0.083229,1.001462,0.086773,91.944616,141.918525,"Hidden Size=[10], regularizer=0.2, learning_ra..."
1,model_22_9_1,0.998952,0.789111,0.989611,0.995397,0.992373,0.007008,1.410218,0.031711,0.011621,0.021666,0.345269,0.083712,1.001479,0.087276,91.921496,141.895405,"Hidden Size=[10], regularizer=0.2, learning_ra..."
2,model_22_9_2,0.998941,0.789076,0.989512,0.995296,0.992275,0.007083,1.410452,0.032013,0.011876,0.021944,0.348293,0.084158,1.001495,0.087741,91.900221,141.874130,"Hidden Size=[10], regularizer=0.2, learning_ra..."
3,model_22_9_3,0.998930,0.789044,0.989422,0.995204,0.992186,0.007152,1.410666,0.032287,0.012108,0.022198,0.351016,0.084571,1.001510,0.088171,91.880677,141.854585,"Hidden Size=[10], regularizer=0.2, learning_ra..."
4,model_22_9_4,0.998921,0.789015,0.989341,0.995121,0.992105,0.007216,1.410860,0.032535,0.012319,0.022427,0.353459,0.084949,1.001524,0.088566,91.862813,141.836722,"Hidden Size=[10], regularizer=0.2, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,model_35_6_8,0.952422,0.789650,0.857897,0.975272,0.949503,0.318154,1.406612,0.365794,0.133566,0.249680,0.982482,0.564052,1.039375,0.588065,108.290437,172.890855,"Hidden Size=[13], regularizer=0.05, learning_r..."
1635,model_25_7_17,0.952368,0.824435,0.776074,0.957172,0.942947,0.318518,1.174005,0.339095,0.372529,0.355812,0.973503,0.564374,1.054437,0.588400,92.288155,147.137567,"Hidden Size=[11], regularizer=0.2, learning_ra..."
1641,model_5_8_24,0.952328,0.781975,0.863935,0.834210,0.920597,0.318781,1.457937,0.189458,0.115286,0.152372,0.777871,0.564607,2.144121,0.588643,52.286502,82.758397,"Hidden Size=[6], regularizer=0.2, learning_rat..."
1644,model_27_8_5,0.952292,0.812027,0.974910,0.908052,0.967207,0.319025,1.256977,0.125531,0.130929,0.128230,1.154777,0.564823,1.054524,0.588869,92.284969,147.134381,"Hidden Size=[11], regularizer=0.05, learning_r..."
